In [1]:
import numpy as np #Import relevant packages
import pandas as pd
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import glob
import csv
import matplotlib.pyplot as plt

def unpack_data(filepath): #Returns a dictionary where each item is a df containing one run. One dict per generation.
    all_files = glob.glob(filepath + "/*.csv")
    name_list = []
    datadict = {}
    for f in all_files:
        name_list.append(f[77:-4])
    for i,n in enumerate(name_list):
        datadict[n] = pd.read_csv(all_files[i], header=0)
    for k, v in datadict.items():
        v.drop(columns=["Unnamed: 0", "volume", "spread", "10_MA", "50_MA"], inplace=True)
    return datadict

def format_data(datadict, window): #Turn dictionaries of data into features and samples lists
    x_data = []
    y_data = []
    for k, v in datadict.items():
        for i in range(len(v)):
            if i >= window:
                y_data.append(v.iloc[i]["trading_price"])
                xi = v.iloc[i-window:i].to_numpy()
                x_data.append(xi)
    x_data = np.array(x_data)
    y_data = np.array(y_data)
    y_data = np.reshape(y_data, (y_data.shape[0], 1))
    return x_data, y_data

def normalise_data(xarray, yarray): #Take the natural log and then normalise prices
    yarray_log = np.log(yarray)
    xarray_log = np.log(xarray)
    xsc = StandardScaler()
    instances, timesteps, features = xarray_log.shape
    xarray_log = np.reshape(xarray_log, (-1, features))
    xarray_norm = xsc.fit_transform(xarray_log)
    xarray_norm = np.reshape(xarray_norm, (instances, timesteps, features))
    ysc = StandardScaler().fit(yarray_log)
    yarray_norm = ysc.transform(yarray_log)
    return xarray_norm, yarray_norm, xsc, ysc

def split_data(xarray, yarray, trainratio): #Split the data into training and test sets
    train_len = int(len(xarray)*trainratio)
    test_len = len(xarray) - train_len
    x_train, y_train, x_test, y_test = xarray[:train_len], yarray[:train_len], xarray[train_len:], yarray[train_len:]
    return x_train, y_train, x_test, y_test

In [2]:
gen34 = unpack_data("/Users/karangarg/Documents/Year 3 Modules/EC331/Code/rae_repo/simulations/gen34_sims/data")

In [3]:
X35, y35 = format_data(gen34, 20)
print(X35.shape)
print(y35.shape)

(29993, 20, 2)
(29993, 1)


In [4]:
X35_train, y35_train, X35_test, y35_test = split_data(X35, y35, 0.8)

In [5]:
X35_train, y35_train, X35scale, y35scale = normalise_data(X35_train, y35_train)
X35_test, y35_test, _, _ = normalise_data(X35_test, y35_test)

In [6]:
print(X35scale.mean_)
print(X35scale.var_)
print(y35scale.mean_)
print(y35scale.var_)

[4.64240638 4.80279299]
[0.23289096 0.01653679]
[4.64465301]
[0.2478805]


In [7]:
lstm35=keras.Sequential() #Set up the architecture of the model

lstm35.add(layers.LSTM(units=16, input_shape=(X35_train.shape[1], X35_train.shape[2])))
lstm35.add(layers.Dropout(0.2))

lstm35.add(layers.Dense(units=1))

print(lstm35.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 16)                1216      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 1,233
Trainable params: 1,233
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
lstm35.compile(optimizer='nadam', loss='mean_squared_error') #Compile and train the model

history35 = lstm35.fit(X35_train, y35_train, epochs = 10, batch_size = 32, validation_split=0.2, shuffle=False)

Train on 19195 samples, validate on 4799 samples
Epoch 1/10
19195/19195 [==============================] - 34s 2ms/sample - loss: 0.2363 - val_loss: 0.0214
Epoch 2/10
19195/19195 [==============================] - 25s 1ms/sample - loss: 0.0288 - val_loss: 0.0043
Epoch 3/10
19195/19195 [==============================] - 21s 1ms/sample - loss: 0.0202 - val_loss: 0.0048
Epoch 4/10
19195/19195 [==============================] - 31s 2ms/sample - loss: 0.0188 - val_loss: 0.0039
Epoch 5/10
19195/19195 [==============================] - 26s 1ms/sample - loss: 0.0186 - val_loss: 0.0225
Epoch 6/10
19195/19195 [==============================] - 25s 1ms/sample - loss: 0.0176 - val_loss: 0.0048
Epoch 7/10
19195/19195 [==============================] - 27s 1ms/sample - loss: 0.0170 - val_loss: 0.1023
Epoch 8/10
19195/19195 [==============================] - 26s 1ms/sample - loss: 0.0170 - val_loss: 0.0387
Epoch 9/10
19195/19195 [==============================] - 28s 1ms/sample - loss: 0.0177 - val_l

In [9]:
lstm35.save("/Users/karangarg/Documents/Year 3 Modules/EC331/Code/rae_repo/lstms/lstm35")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /Users/karangarg/Documents/Year 3 Modules/EC331/Code/rae_repo/lstms/lstm35/assets


In [10]:
plt.figure(figsize=(15,6))
plt.plot(history35.history["loss"], label="train")
plt.plot(history35.history["val_loss"], label="validation")
plt.legend()

In [11]:
y35_pred = lstm35.predict(X35_test)
y35_train_inv = y35scale.inverse_transform(y35_train.reshape(1, -1))
y35_test_inv = y35scale.inverse_transform(y35_test.reshape(1, -1))
y35_pred_inv = y35scale.inverse_transform(y35_pred)

In [12]:
plt.figure(figsize=(15,6))
plt.plot(y35_test_inv.flatten(), label="Test data")
plt.plot(y35_pred_inv.flatten(), label="Model prediction")
plt.legend()